In [1]:
from chain_builder import get_sentence
import cPickle as pickle
import tweepy
from datetime import date, timedelta
from keys import keys

In [2]:
with open('Pickles/forward_dict.pkl') as f:
    f_dict = pickle.load(f)

with open('Pickles/reversed_dict.pkl') as f:
    r_dict = pickle.load(f)

In [3]:
CONSUMER_KEY = keys['consumer_key']
CONSUMER_SECRET = keys['consumer_secret']
ACCESS_TOKEN = keys['access_token']
ACCESS_TOKEN_SECRET = keys['access_token_secret']

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

tweet_replies = []

today = date.today().strftime("%Y-%m-%d")
yesterday = date.today() - timedelta(1)

# ******************************************************
# Queries Popular Tweets directed at Candidates or using
# well-known Hashtags and responds to all queried tweets.
# ******************************************************
hashtags = ["#FeelTheBern OR #ImWithHer OR #MakeAmericaGreatAgain OR #Trump2016 \
OR @BernieSanders OR @HillaryClinton OR @realDonaldTrump"]

top_tweets = tweepy.Cursor(api.search, q=hashtags, count=100, result_type='popular', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in top_tweets:
    if status.favorite_count > 1000:
        tweet_replies.append(status)

# ***************************************************
# Queries Tweets directed at or in reply to
# FeelTheBern_Bot and responds to all queried tweets.
# ***************************************************
bernbot_query = "to:FeelTheBern_Bot OR @FeelTheBern_Bot"

bernbot_tweets = tweepy.Cursor(api.search, q=bernbot_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in bernbot_tweets:
    tweet_replies.append(status)

# ************************************************
# Queries Tweets from Presidential Candidates and
# responds to those with greater than 500 likes.
# ************************************************
candidate_query = "from:BernieSanders OR from:HillaryClinton OR from:realDonaldTrump"

candidate_tweets = tweepy.Cursor(api.search, q=candidate_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in candidate_tweets:
    if status.favorite_count > 500:
        tweet_replies.append(status)


# DROPS ANY DUPLICATE TWEETS SO IT DOES NOT RESPOND TWICE
tweet_replies = set(tweet_replies)

In [5]:
best_tweets = []
for i, tweet in enumerate(tweet_replies):
    resp_length = 141
    text = tweet.text
    sn = tweet.user.screen_name

    while resp_length > 140:
        response = get_sentence(text, f_dict, r_dict, randomness = 1) 
        response = "@{0} {1}".format(sn, response)
        resp_length = len(response)
    
    print i, '----', tweet.id
    print '@', sn, text
    print response
    print '*************'
    best_tweets.append([response, tweet.id])

0 ---- 726776806519394304
@ CLewandowski_ Thank you Indiana! #Trump2016 will not let you down! #MAGA NBC/WSJ https://t.co/CcNRSbTaHQ
@CLewandowski_ Because let me tell you.
*************
1 ---- 726886687276699648
@ realDonaldTrump Thank you Fort Wayne, Indiana!
#Trump2016 #INPrimary https://t.co/mDGrmMmk5T
@realDonaldTrump Well, thank you.
*************
2 ---- 726569008011444224
@ FoxNews Poll: @realDonaldTrump leads in Indiana with 41%. https://t.co/ZNA2Qhyzvc
@FoxNews Senator Sanders, you're the only one that leads Hillary Clinton in five states that were won by President Obama.
*************
3 ---- 726867935516565504
@ HillaryClinton "She’s battle tested, she’s got a dirty uniform, and that’s who you want fighting for your kids’ future.” https://t.co/tQx1xcF3ew
@HillaryClinton And then you know what you would do as President, and if I haven't been during the time I went back, it got done and here's.
*************
4 ---- 726773343664377856
@ HillaryClinton Raise the minimum wage.
Ens

In [7]:
best_list = [21]
for i, tweet in enumerate(best_tweets):
    if i in best_list:
        print i 
        print tweet[0], tweet[1]
        s = api.update_status(tweet[0], tweet[1])

21
@ABCPolitics I understand, I understand why they feel that way, she could spend her own money with her face. 726597323552333825
